# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096885


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:40,  1.44s/it]

Rendering models:  10%|█         | 3/30 [00:03<00:28,  1.05s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:41,  1.59s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:28,  1.16s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:11<00:00,  5.11it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  6.17it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  5.68it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  5.31it/s]

not-logged-in-3b0b929f4475fd276561    0.000208
ethanth                               0.000113
nurenai                               0.049399
DevionJ                               0.000143
framosuw                              0.008116
Quarkgc                               0.000697
jnarayanbvg                           0.029696
Jonas_Cross                           0.000417
gracie_solveig                        0.000314
jaksonA                               2.508176
becca1029                             0.000214
Theaker                               0.005174
not-logged-in-468edef11712cfc1e24c    0.024671
pangeli5                              0.000086
froggler                              0.000102
not-logged-in-04ed9ae880f18762cd9e    0.055366
Lavadude                              0.037950
BrowardDaniel                         0.026870
slambier                              0.000136
kgenere                               0.000300
vceccare                              0.001389
Fazal.H      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())